### Importing Required Libraries

This cell imports various Python libraries needed for data processing, machine learning, and text analysis.

In [5]:
import pandas as pd
import re
from time import time
from rapidfuzz import process, fuzz
from deep_translator import GoogleTranslator
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import os
import urllib.request
import zipfile

### Loading the Trained Model and Vectorizer

In [6]:
# Define paths
model_dir = "models"
model_file = os.path.join(model_dir, "logistic_regression_model.pkl")
vectorizer_file = os.path.join(model_dir, "tfidf_vectorizer.pkl")
zip_url = "https://github.com/Marwan513/Pharmaceutical-product-matching/releases/download/v1.0/models.zip"
zip_file = "models.zip"

# Check if models exist, if not, download and extract
if not (os.path.exists(model_file) and os.path.exists(vectorizer_file)):
    print("Models not found. Downloading...")
    urllib.request.urlretrieve(zip_url, zip_file)
    print("Download complete. Extracting...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall()
    os.remove(zip_file)
    print("Extraction complete.")
else:
    print("Models already exist. Skipping download.")

# Load the model and vectorizer
model = joblib.load(model_file)
vectorizer = joblib.load(vectorizer_file)

print("Model and vectorizer loaded successfully!")

Models not found. Downloading...
Download complete. Extracting...
Extraction complete.
Model and vectorizer loaded successfully!


### Function to Preprocesses the dataset

In [7]:
def preprocess_function(dataset, master_file):
    """
    Preprocesses the dataset by translating English seller item names to Arabic and normalizing the Arabic text.

    This function performs the following:
    1. Translates English product names to Arabic using fuzzy matching with a master file and Google Translator as a fallback.
    2. Normalizes the Arabic text by removing diacritics, unwanted words, and handling various common variations in Arabic.
    3. Transforms Arabic numbers to English numbers.

    Args:
        dataset (pandas.DataFrame): The dataset containing seller item names to be processed.
        master_file (pandas.DataFrame): A master file containing product names in English and their corresponding Arabic translations.

    Returns:
        pandas.DataFrame: The dataset with a new column 'processed_seller_item_name' containing the processed Arabic names.
    """
    # Arabic number conversion dictionary
    arabic_to_english_numbers = str.maketrans("٠١٢٣٤٥٦٧٨٩", "0123456789")

    # Create a dictionary for English-to-Arabic translation from the Master File
    translation_dict = dict(zip(master_file["product_name"].astype(str).str.lower(), master_file["product_name_ar"].astype(str)))

    # List of English product names from the Master File for fuzzy matching
    master_names_en = list(translation_dict.keys())

    # Function to check if text contains English characters
    def contains_english(text):
        return bool(re.search("[A-Za-z]", text))

    # Function to translate English to Arabic (with retry logic)
    def translate_to_arabic(text):
        if contains_english(text):   # Only translate if it contains English
            text_lower = text.lower().strip()
            text = text.replace('.', ' ')

            # Find the closest match from the Master File
            match, score, _ = process.extractOne(text_lower, master_names_en, scorer=fuzz.ratio)

            # If match is strong (90% similarity or higher), use the Master File translation
            if score >= 50:
                return translation_dict[match]

            try:
                translated_text = GoogleTranslator(source="english", target="arabic").translate(text_lower)
                if translated_text and not contains_english(translated_text):
                    return translated_text
                else:
                    print(f"Translation did not return Arabic text for: ({text}) and instead return ({translation_dict[match]}) and score : {score}")
                    return text  # Return original if translation is not in Arabic
            except Exception as e:
                print(f"Translation failed for {text}: {e}")
        return text  # Return original if translation fails

    # Function to remove diacritics and normalize Arabic text
    def remove_diacritics(text):
        arabic_diacritics = re.compile("[\u064B-\u0652]")
        return re.sub(arabic_diacritics, "", text)

    # Function to remove specific unwanted words ("جديد" with variations & "سعر")
    def remove_unwanted_words(text):
        text = re.sub(r"جدي+د", "", text)  # Remove "جديد" with varying "ي" count
        text = re.sub(r"\bسعر\b", "", text)  # Remove exact match "سعر"
        text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
        return text

    def normalize_arabic(text):
        text = str(text).strip()
        text = remove_diacritics(text)
        text = text.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا")  # Normalize Alef
        text = text.replace("ى", "ي").replace("ة", "ه").replace("ٱ", "ا")  # Normalize common variations
        text = text.replace("ؤ", "و").replace("ئ", "ي")  # Normalize more variations
        text = re.sub(r"[^\u0600-\u06FF0-9 %\\/]", "", text)  # Remove non-Arabic characters except numbers
        text = text.translate(arabic_to_english_numbers)  # Convert Arabic numbers to English
        text = re.sub(r"(\d+)", r" \1 ", text).strip()  # Add spaces before and after numbers
        text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
        text = re.sub(r"ـ+", "", text)  # Remove extensions in words
        text = remove_unwanted_words(text)  # Remove "جديد" variations and "سعر"
        return text

    # Translate only English seller names to Arabic
    dataset["processed_seller_item_name"] = dataset["seller_item_name"].astype(str).apply(translate_to_arabic)

    # Normalize translated Arabic names
    dataset["processed_seller_item_name"] = dataset["processed_seller_item_name"].apply(normalize_arabic)

    return dataset

### **process_file Function Documentation**

**Overview**:
The `process_file` function processes an input Excel file containing a dataset and a master file. It performs translation, similarity matching, and SKU mapping, then saves the results to an output Excel file.

**Functionality**:
This function follows these key steps:
1. **Load Data:** Reads the input Excel file containing two sheets: `"Dataset"` and `"Master File"`.
2. **Validate Columns:** Ensures that the necessary columns exist in both sheets.
3. **Preprocessing:** Applies the provided preprocessing function to clean and normalize seller item names.
4. **Compute Similarity:** Uses a machine learning model to predict the most relevant marketplace product name and compute a similarity score.
5. **Find SKU:** Uses fuzzy matching to identify the best SKU from the master file.
6. **Save Output:** Writes the processed dataset back to an Excel file with similarity scores and matched SKUs.

**Output**:
The processed dataset includes:
- `marketplace_product_name_ar`: Predicted marketplace name.
- `similarity`: Cosine similarity score.
- `confidence`: Confidence level of the match.
- `sku`: Mapped SKU based on fuzzy matching.


In [10]:
def process_file(input_file, master_file, model, vectorizer, preprocess_func, output_file="output.xlsx"):
    """
    Processes an input Excel file containing a dataset and a master file, performs translation and similarity matching,
    and saves the results to an output Excel file.

    This function performs the following steps:
    1. Loads the input Excel file with two sheets: "Dataset" and "Master File".
    2. Ensures that the required columns are present in the dataset and master file.
    3. Preprocesses the seller item names using the provided preprocessing function.
    4. Computes similarity scores between seller item names and predicted marketplace names.
    5. Uses fuzzy matching to find the best SKU from the master file.
    6. Saves the processed dataset with similarity and SKU information to an output Excel file.

    Args:
        input_file (str): The input Excel file containing two sheets ("Dataset" and "Master File").
        model (sklearn.model): The trained machine learning model used for marketplace name predictions.
        vectorizer (sklearn.feature_extraction.text.TfidfVectorizer): The TF-IDF vectorizer used for feature extraction.
        preprocess_func (function): The preprocessing function used to process seller item names.
        output_file (str, optional): The name of the output Excel file. Default is "output.xlsx".

    Returns:
        None
    """

    # Load the Excel files
    df = pd.read_excel(input_file)  # Replace with the actual file path
    master_df = pd.read_excel(master_file)

    # Ensure seller_item_name exists in input_file
    if 'seller_item_name' not in df.columns:
        raise ValueError(f"Column 'seller_item_name' not found in dataset sheet.")

    # Ensure product_name_ar exists in master_file
    if 'product_name_ar' not in master_df.columns:
        raise ValueError("Column 'product_name_ar' not found in master_file sheet.")

    # Preprocess seller item names
    df = preprocess_func(df,master_df)

    # Function to compute similarity (based on your function)
    def compute_similarity(seller_name):

        # Transform seller name to TF-IDF vector
        seller_vector = vectorizer.transform([seller_name])

        # Predict the most likely marketplace name using the TF-IDF vector
        predicted_name = model.predict(seller_vector)[0]

        # Transform predicted name to TF-IDF vector
        predicted_vector = vectorizer.transform([predicted_name])

        # Compute cosine similarity
        similarity_score = cosine_similarity(seller_vector, predicted_vector)[0, 0]

        # Set confidence levels
        if similarity_score < 0.2:
            matched_name = "Not Found"
            confidence = "Unknown"
        else:
            matched_name = predicted_name
            confidence = "High" if similarity_score > 0.8 else "Medium" if similarity_score > 0.6 else "Low"


        return matched_name, similarity_score, confidence

    # Apply matching function
    df[['marketplace_product_name_ar', 'similarity', 'confidence']] = df['processed_seller_item_name'].apply(
        lambda name: pd.Series(compute_similarity(name))
    )

    # Use fuzzy matching to find the best SKU
    product_names = master_df['product_name_ar'].tolist()
    sku_dict = dict(zip(master_df['product_name_ar'], master_df['sku']))

    def find_best_sku(marketplace_name):
        if marketplace_name == "Not Found":
            return "Not Found"
        match, score, _ = process.extractOne(marketplace_name, product_names, scorer=fuzz.token_sort_ratio)
        return sku_dict.get(match, "Not Found") if score > 70 else "Not Found"

    df['sku'] = df['marketplace_product_name_ar'].apply(find_best_sku)

    # Drop duplicate columns & temp column
    df.drop(columns=['processed_seller_item_name'], inplace=True)

    # Save df to an Excel file
    df.to_excel("output.xlsx", sheet_name="dataset", index=False)

    print(f"Processed file saved as {output_file}")


### **Execution Time Analysis for process_file Function**

measures the execution time of the `process_file` function while processing an Excel file.

In [11]:
start_time = time()

# Process the test file
process_file("test file.xlsx", 'master_file.xlsx', model, vectorizer, preprocess_function, output_file="output.xlsx") # edit the path if you want to test on another test data

# Calculate total execution time in milliseconds
execution_time = (time() - start_time) * 1000

# Get the actual number of samples processed
num_samples = pd.read_excel("test file.xlsx", sheet_name="Dataset").shape[0]

# Calculate average time per sample dynamically
average_time_per_sample = execution_time / num_samples if num_samples > 0 else 0

# Print execution results
print(f"Total Execution Time: {round(execution_time, 2)} ms")
print(f"Average Time per Sample: {average_time_per_sample:.2f} ms")



Processed file saved as output.xlsx
Total Execution Time: 71695.02 ms
Average Time per Sample: 71.70 ms
